In [1]:
import pandas as pd
import datetime as dt

# configs:

# this is the directory in which the daily data files are locations from Johns Hopkins
daily_data_dir = '/Users/robrobinson/develop/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'

# where to place our output csvs
output_dir = '/Users/robrobinson/develop/covid-19-data-visualization/deployment/app_src/output/'

# where to look for local csv data
local_data_dir = '/Users/robrobinson/develop/covid-19-data-visualization/deployment/app_src/data/'

# get a list of dates for which we use the older headers/labels
model_one_dates = pd.date_range(start="2020-03-01", end="2020-03-21").to_pydatetime().tolist()

# get a list of dates from March 3rd, to today
all_dates = pd.date_range(start="2020-03-10", end=dt.datetime.today()).to_pydatetime().tolist()

# initial load of just states
all_data = pd.read_csv(local_data_dir + 'us_states.csv')

In [2]:
for date in all_dates:

    # create a string version of the date for use in file creation and labels:
    date_str = date.strftime("%m-%d-%Y")

    # open the proper file
    daily_csv_data_full_file_path = daily_data_dir + date_str + '.csv'

    try:

        # use older header labels
        if date in model_one_dates:

            df = pd.read_csv(daily_csv_data_full_file_path, skipinitialspace=False,
                             usecols=['Province/State', 'Country/Region', 'Confirmed', 'Deaths'])

        # use new header labels
        else:
            df = pd.read_csv(daily_csv_data_full_file_path, skipinitialspace=False,
                             usecols=['Province_State', 'Country_Region', 'Confirmed', 'Deaths'])

    except OSError as e:
        # print('Note: '+date_str+'.csv was not found ... ' + e.strerror)
        continue  # will skip the rest of the block and move to next file

    df.rename(columns={'Confirmed': 'Confirmed-' + date_str}, inplace=True)
    df.rename(columns={'Deaths': 'Deaths-' + date_str}, inplace=True)

    df.rename(columns={'Province/State': 'Province_State'}, inplace=True)
    df.rename(columns={'Country/Region': 'Country_Region'}, inplace=True)

    # Filter for just US Territories:
    us_data = df.query("Country_Region == 'US'")

    # Apply grouping, but don't use groupby field as the index:
    state_data = us_data.groupby('Province_State', as_index=False)

    # create new dataframe using aggregated group by...
    tmp_df = state_data.sum()

    # merge the current dataframe into the master
    all_data = pd.merge(all_data, tmp_df, on='Province_State')

In [3]:
def get_deaths(input_list):

    to_return = []

    for line in input_list:

        if line == 'Province_State':
            to_return.append(line)

        if 'Deaths-' in line:
            to_return.append(line)

    return to_return


def get_confirmed(input_list):

    to_return = []

    for line in input_list:

        if line == 'Province_State':
            to_return.append(line)

        if 'Confirmed-' in line:
            to_return.append(line)

    return to_return

all_columns = list(all_data.columns.values)

deaths_columns = get_deaths(all_columns)

confirmed_columns = get_confirmed(all_columns)

df_confirmed = all_data[confirmed_columns]
df_deaths = all_data[deaths_columns]

In [4]:
df_confirmed

,Province_State,Confirmed-03-10-2020,Confirmed-03-11-2020,Confirmed-03-12-2020,Confirmed-03-13-2020,Confirmed-03-14-2020,Confirmed-03-15-2020,Confirmed-03-16-2020,Confirmed-03-17-2020,Confirmed-03-18-2020,Confirmed-03-19-2020,Confirmed-03-20-2020,Confirmed-03-21-2020,Confirmed-03-22-2020,Confirmed-03-23-2020,Confirmed-03-24-2020,Confirmed-03-25-2020,Confirmed-03-26-2020,Confirmed-03-27-2020,Confirmed-03-28-2020
0,Arizona,6,9,9,9,12,13,18,20,27,45,78,118,152,235,326,401,508,665,773
1,Arkansas,0,1,6,6,12,16,22,22,33,62,96,122,165,192,219,280,335,381,409
2,California,144,177,221,282,340,426,557,698,751,952,1177,1364,1646,2108,2538,2998,3899,4657,5095
3,Colorado,15,34,45,49,101,131,160,160,184,277,363,390,591,704,723,1021,1430,1433,1740
4,Connecticut,2,3,5,11,22,24,30,68,68,159,194,194,327,415,618,875,1012,1291,1524
5,Delaware,0,1,1,4,6,7,8,16,19,30,38,45,56,68,104,119,130,163,214
6,District of Columbia,5,10,10,10,10,16,22,22,31,40,71,77,102,120,141,187,231,271,304
7,Florida,15,28,35,50,76,115,155,216,314,417,563,659,1004,1227,1412,1682,2357,2900,3763
8,Georgia,17,23,31,42,66,99,121,146,199,287,420,507,621,772,1026,1247,1525,2000,2366
9,Hawaii,2,2,2,2,4,6,7,10,14,16,26,37,53,56,90,91,95,106,149


In [5]:
df_deaths

,Province_State,Deaths-03-10-2020,Deaths-03-11-2020,Deaths-03-12-2020,Deaths-03-13-2020,Deaths-03-14-2020,Deaths-03-15-2020,Deaths-03-16-2020,Deaths-03-17-2020,Deaths-03-18-2020,Deaths-03-19-2020,Deaths-03-20-2020,Deaths-03-21-2020,Deaths-03-22-2020,Deaths-03-23-2020,Deaths-03-24-2020,Deaths-03-25-2020,Deaths-03-26-2020,Deaths-03-27-2020,Deaths-03-28-2020
0,Arizona,0,0,0,0,0,0,0,0,0,0,0,1,2,2,5,6,8,13,15
1,Arkansas,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,3,5
2,California,2,3,4,4,5,6,7,12,13,18,23,24,30,39,50,65,81,94,110
3,Colorado,0,0,0,0,1,1,1,2,2,4,4,4,6,7,8,16,19,27,31
4,Connecticut,0,0,0,0,0,0,0,0,0,2,3,4,8,10,12,19,21,27,33
5,Delaware,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,5
6,District of Columbia,0,0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,3,3,4
7,Florida,2,2,2,2,3,4,5,6,7,9,10,13,13,18,18,23,29,35,54
8,Georgia,0,0,1,1,1,1,1,1,3,10,13,14,25,25,32,40,48,64,69
9,Hawaii,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
